1. Euclidian distance function [4 points]
============
The squared Euclidean distance between two D-dimensional vectors x and z is given by
$\newcommand{\norm}[1]{\left\lVert#1\right\rVert}$

$\norm{\mathbf{x} - \mathbf{z}}_{2}^{2} = (\mathbf{x} - \mathbf{z})^\top(\mathbf{x} - \mathbf{z}) = \sigma_{d=1}^{D}(X_d - Z_d)^2$

In [190]:
import tensorflow as tf;

def euclidianDist(a, b):
    x_z = (tf.expand_dims(a,0) - tf.expand_dims(b,1))
    return tf.reduce_sum(tf.multiply(x_z, x_z), 2)

# Alternative Solution
#def distanceFunc(X, Z):
#    return tf.reduce_sum((tf.expand_dims(X, 2) - tf.expand_dims(tf.transpose(Z), 0))**2, 1)

sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

a = np.array([[0,0],[0,1],[0,2]])
b = np.array([[1,0],[1,1],[1,2]])
print('D_euc = ')
print(a)
print(b)
print(sess.run(euclidianDist(a,b)))

D_euc = 
[[0 0]
 [0 1]
 [0 2]]
[[1 0]
 [1 1]
 [1 2]]
[[1 2 5]
 [2 1 2]
 [5 2 1]]


2. Making Predictions for Regression [4 points]
============


In [257]:
# Import data
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py

k = 2

np.random.seed(521)
Data = np.linspace(1.0 , 10.0 , num =100) [:, np. newaxis]
Target = np.sin( Data ) + 0.1 * np.power( Data , 2) \
         + 0.5 * np.random.randn(100 , 1)
randIdx = np.arange(100)
np.random.shuffle(randIdx)
trainData, trainTarget  = Data[randIdx[:80]], Target[randIdx[:80]]
validData, validTarget = Data[randIdx[80:90]], Target[randIdx[80:90]]
testData, testTarget = Data[randIdx[90:100]], Target[randIdx[90:100]]

# Draw the plot
line = plt.figure()
plt.title('k-NN regression on data1D, k = ' + str(k))
plt.plot(trainData, trainTarget, "x")
plt.plot(validData, validTarget, "x")
plt.plot(testData, testTarget, "x")
plt.legend(['Train', 'Valid', 'Test'])
plt.show()

# def pickKNearest(Distance, K=50):
#      numTrainData = tf.shape(Distance)[1]
#      dist_k, ind_k = tf.nn.top_k(-Distance, k = K)
#      R = tf.reduce_sum(tf.to_float(tf.equal(tf.expand_dims(ind_k,2),
#                               tf.reshape(tf.range(numTrainData), [1,1,-1]))),1)
#      return R/tf.to_float(K)

def pickKNearestNeighbours(DistMatrix, k):
    length = tf.shape(DistMatrix)[0]
    
    values, indices = tf.nn.top_k(-DistMatrix, k)
    range = tf.range(length)
    
    rangeblock = tf.expand_dims(tf.expand_dims(range, 0),0)
    indexblock = tf.expand_dims(indices, 2)
    
    truth_matrix = tf.reduce_sum(tf.to_float(tf.equal(rangeblock, indexblock)),1)

    return tf.scalar_mul(1/k,truth_matrix)

def predKNN(R, trainY):
    return tf.matmul(R, trainY)
    
## begin building graph
trainX = tf.placeholder(tf.float32, [None, 1], name=’input_x’)
trainY = tf.placeholder(tf.float32, [None, 1], name=’input_y’)
newX = tf.placeholder(tf.float32, [None, 1], name=’new_x’)
newY = tf.placeholder(tf.float32, [None, 1], name=’new_y’)
K = tf.placeholder("int32", name=’K’)
predY = predKNN(pickKNearest(distanceFunc(newX, trainX), K = K), trainY) MSE = tf.reduce_mean(tf.reduce_sum((predY - newY)**2, 1))
sess = tf.InteractiveSession()
## end building graph
X = np.linspace(0.0,11.0,num=1000)[:,np.newaxis] # for plotting import matplotlib.pyp # Find the nearest k neighbours:
kvec = [1,3,5,50]
mse_valid_list = []
mse_test_list = []
for kc in kvec:
    mse_valid = sess.run(MSE, feed_dict={trainX:trainData, trainY:trainTarget,
                                      newX:validData, newY:validTarget, K:kc})
    mse_test = sess.run(MSE, feed_dict={trainX:trainData, trainY:trainTarget,
                                      newX:testData, newY:testTarget, K:kc})
    mse_valid_list.append(mse_valid)
    mse_test_list.append(mse_test)
    print("K=%d\t validation MSE: %f, test MSE: %f"%(kc,mse_valid,mse_test))
    yp = sess.run(predY, feed_dict={trainX:trainData, trainY:trainTarget,
                                      newX:X, K:kc})
    plt.figure(kc+1)
    plt.plot(trainData,trainTarget,’.’)
    plt.plot(X,yp,’-’)
    plt.title("k-NN regression on datat1D, k=%d"%kc)
    plt.show()
k_best = kvec[np.argmin(mse_valid_list)]
print("best K using validation set is K=%d"%(k_best))

print(sess.run(pickKNearestNeighbours(euclidianDist(a,b),3)))

SyntaxError: invalid character in identifier (<ipython-input-257-f7585b2570d9>, line 50)